In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load the data from the UN FAO DAD-IS DB
# Domestic Animal Diversity Information System (DAD-IS)
dad_is_df = pd.read_csv('../data/102821_FAO_DAD-IS_export.csv')
dad_is_df = dad_is_df.replace({'Duck (domestic)': 'Duck', 'Duck(domestic) Muscovy duck': 'Muscovy Duck',
                               'Goose (domestic)': 'Goose', 'Dromedary Bactrian Camel': 'Bactrian Camel',
                               'Vicuña': 'Vicuna', 'Yak (domestic)': 'Yak'})


dad_is_df.columns.tolist()

['Country',
 'ISO3',
 'Specie',
 'Breed/Most common name',
 'Description Of Specific Uses',
 'Lang',
 'Description',
 'Transboundary name',
 'Other name',
 'Uses',
 'Additional information',
 'Additional information comments',
 'Efabis cultural role comment',
 'Efabis cultural value',
 'Adaptability to specific environment',
 'Specific resistance or tolerance',
 'Specific reproductive characteristic',
 'Special characteristic of product',
 'Other special qualities',
 'Reference for special qualities',
 'Efabis genetic features',
 'Efabis environmental role',
 'Efabis adaptability to marginal land',
 'Body conformation',
 'Coat description',
 'Coat quality',
 'Comb type',
 'Skin colour',
 'Shank and foot colour',
 'Plumage colour',
 'Pattern within feather',
 'Avian classification',
 'Color comments',
 'Efabis main colour',
 'Efabis skin colour',
 'Number of horns males',
 'Number of horns females',
 'Horn shape size and comments',
 'Wither height males',
 'Wither height females',
 'Wei

In [3]:
cols_of_interest = ['Weight males', 'Weight females', 'Birth weight males', 'Birth weight females',
                    'Age maturity males', 'Age maturity females', 'Age breeding males', 'Age breeding females',
                    'Age first parturition AVG', 'Age first parturition MIN', 'Age first parturition MAX',
                    'Parturition interval AVG', 'Parturition interval MIN', 'Parturition interval MAX',
                    'Length productive life', 'Daily gain', 'Carcass weight', 'Dressing percentage',
                    'Litter size AVG', 'Litter size MIN', 'Litter size MAX']
# List of units for columns above, gleaned from GUI interface here:
# https://www.fao.org/dad-is/browse-by-country-and-species/en/
units = ['kg']*4 + ['months']*7 + ['day']*3 + ['year', 'g', 'kg', '%'] + ['number']*3
new_colnames = ['{0} ({1})'.format(c, u) for c, u in zip(cols_of_interest, units)]

# Zero values are non-sensical - should be NaN for log-scale plotting
for c in cols_of_interest:
    mask = dad_is_df[c] == 0
    dad_is_df.loc[mask, c] = np.nan
    
# Drop everything that is missing data in all columns of interest. 
dad_is_df = dad_is_df.dropna(axis=0, how='all', subset=cols_of_interest)
    
# Calculate the number of young per year from the litter size and interval between litters.
# NOTE: not 100% sure this is what the "Parturition interval" means
litter_size = dad_is_df['Litter size AVG']

# Have to convert 0 interval into NaN otherwise we get infinite young per year
litter_interval = dad_is_df['Parturition interval AVG']

young_per_year = litter_size / (litter_interval/365)
dad_is_df['Young per year AVG (number)'] = young_per_year

mean_adult_weight_kg = dad_is_df[['Weight males', 'Weight females']].mean(axis=1)
dad_is_df['Adult weight AVG (kg)'] = mean_adult_weight_kg
dad_is_df['Adult weight AVG (g)'] = mean_adult_weight_kg*1000

mean_birth_weight_kg = dad_is_df[['Birth weight males', 'Birth weight females']].mean(axis=1)
dad_is_df['Birth weight AVG (kg)'] = mean_birth_weight_kg
dad_is_df['Birth weight AVG (g)'] = mean_birth_weight_kg*1000

neonate_mass_kg = dad_is_df['Birth weight AVG (kg)']
young_per_year = dad_is_df['Young per year AVG (number)']
dad_is_df['Young mass per year AVG (kg)'] = neonate_mass_kg*young_per_year
dad_is_df['Young mass per year AVG (g)'] = neonate_mass_kg*young_per_year*1000

# Put units in the relevant column names
col_replacement_dict = dict((c, nc) for c, nc in zip(cols_of_interest, new_colnames))
dad_is_clean_df = dad_is_df.rename(columns=col_replacement_dict)

# Save a new cleaner file with units for the relevant columns.
dad_is_clean_df.to_csv('../data/FAO_DAD-IS_cleaned.csv')
dad_is_clean_df.shape

(5544, 118)

In [4]:
# Save number of individual breeds for the relevant columns
counts_by_species = dad_is_clean_df.groupby('Specie')['Breed/Most common name'].count()
counts_by_species.name = 'breed_count'
counts_by_species.to_csv('../data/FAO_DAD-IS_breed_counts_by_species.csv')

In [5]:
# Print the species for which there are > 100 breed representatives
mask = counts_by_species > 100
species_enough_data = counts_by_species[mask].index.tolist()

In [6]:
# Save quantiles for the relevant columns for the species that have enough data.
new_colnames += ['Young per year AVG (number)', 'Adult weight AVG (kg)']
mask = dad_is_clean_df.Specie.isin(species_enough_data)
quantiles_by_species = dad_is_clean_df[mask].groupby('Specie').quantile([0.005, 0.05, 0.25, 0.5, 0.75, 0.95, 0.995])[new_colnames]
quantiles_by_species.index = quantiles_by_species.index.set_names(['Specie', 'quantile'])
quantiles_by_species.to_csv('../data/FAO_DAD-IS_quantiles_by_species.csv')
quantiles_by_species.head()

Weight males (kg)  Weight females (kg)  \
Specie  quantile                                           
Buffalo 0.005               307.780               300.00   
        0.050               350.000               320.10   
        0.250               429.275               382.25   
        0.500               520.500               448.25   
        0.750               600.000               494.50   

                  Birth weight males (kg)  Birth weight females (kg)  \
Specie  quantile                                                       
Buffalo 0.005                      16.320                     15.305   
        0.050                      18.012                     16.060   
        0.250                      25.000                     24.200   
        0.500                      30.000                     27.950   
        0.750                      33.000                     30.750   

                  Age maturity males (months)  Age maturity females (months)  \
Specie  quantile                                                               
Buffalo 0.005                           18.22                         20.145   
        0.050                           20.00                         21.000   
        0.250                           24.00                         25.250   
        0.500                           30.00                         30.000   
        0.750                           33.00                         33.405   

                  Age breeding males (months)  Age breeding females (months)  \
Specie  quantile                                                               
Buffalo 0.005                            24.0                         26.660   
        0.050                            24.0                         30.000   
        0.250                            33.8                         32.000   
        0.500                            36.0                         36.000   
        0.750                            45.0                         45.175   

                  Age first parturition AVG (months)  \
Specie  quantile                                       
Buffalo 0.005                                   24.0   
        0.050                                   29.9   
        0.250                                   36.0   
        0.500                                   42.0   
        0.750                                   47.5   

                  Age first parturition MIN (months)  ...  \
Specie  quantile                                      ...   
Buffalo 0.005                                   22.0  ...   
        0.050                                   22.8  ...   
        0.250                                   34.0  ...   
        0.500                                   36.0  ...   
        0.750                                   45.0  ...   

                  Parturition interval MAX (day)  \
Specie  quantile                                   
Buffalo 0.005                             83.245   
        0.050                            427.450   
        0.250                            501.500   
        0.500                            574.000   
        0.750                            645.000   

                  Length productive life (year)  Daily gain (g)  \
Specie  quantile                                                  
Buffalo 0.005                            8.0475         228.125   
        0.050                            8.4750         262.500   
        0.250                           10.7500         361.750   
        0.500                           13.5000         542.500   
        0.750                           15.0000         680.000   

                  Carcass weight (kg)  Dressing percentage (%)  \
Specie  quantile                                                 
Buffalo 0.005                 137.625                   23.737   
        0.050                 161.250                   41.730   
        0.250                 187.500             